In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
from tensorflow.keras.datasets import mnist

In [3]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [17]:
x_train.shape,x_test.shape

((60000, 28, 28), (10000, 28, 28))

The shape of X is 28,28. for passing it for convolution we need 28,28,1

In [5]:
from tensorflow.keras.layers import Dense,Reshape,Flatten,Input,Conv2D,MaxPooling2D

In [6]:
from tensorflow.keras import Sequential

In [7]:
model=Sequential() #encoder

In [8]:
model.add(Input(shape=(28,28)))
model.add(Reshape(target_shape=(28,28,1)))
model.add(Conv2D(32,3,activation="tanh",padding="same")) 
model.add(MaxPooling2D()) #28,28-->14,14
model.add(Conv2D(64,3,activation="tanh",padding="same"))
model.add(MaxPooling2D()) #14,14--->7,7
model.add(Flatten())
model.add(Dense(units=30,activation="sigmoid")) #compressed to 30 fetures

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0         
_________________________________________________________________
dense (Dense)                (None, 30)                9

Everything above is a part of encoder

NOW we need to upsample the 30 features back to 28,28

We used Maxpool for decreasing from 28,28 to 30 features,
to make it back to 28,28 we will use UpSmaple2D

Now we will implement Decoder, output of encoder is the input of decoder

In [10]:
from tensorflow.keras.layers import UpSampling2D

In [11]:
decoder=Sequential()
decoder.add(Input(shape=(30,)))
#to put in conv2D we need to reshape it to 7,7,64 ,3D structure is needed for Conv2D
decoder.add(Dense(units=7*7*64,activation="tanh"))
decoder.add(Reshape(target_shape=(7,7,64)))
decoder.add(Conv2D(64,3,activation="tanh",padding="same"))
decoder.add(UpSampling2D())
decoder.add(Conv2D(32,3,activation="tanh",padding="same"))
decoder.add(UpSampling2D())
decoder.add(Conv2D(1,3,activation="sigmoid",padding="same")) #at this point the image has become 28,28,1
decoder.add(Reshape(target_shape=(28,28)))

decoder.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 3136)              97216     
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 32)        18464     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 1)        

In [12]:
from tensorflow.keras.models import Model

In [13]:
in_layer=Input(shape=(28,28))
encode_layer=model(in_layer) #model-->encoder
decode_layer=decoder(encode_layer)

In [14]:
autoencoder=Model(in_layer,decode_layer)

In [15]:
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
sequential (Sequential)      (None, 30)                112926    
_________________________________________________________________
sequential_1 (Sequential)    (None, 28, 28)            152897    
Total params: 265,823
Trainable params: 265,823
Non-trainable params: 0
_________________________________________________________________


In [18]:
X=x_train.astype(float)/255

In [19]:
autoencoder.compile(optimizer="adam",loss="binary_crossentropy")

In [ ]:
autoencoder.fit(X,X,batch_size=1000,epochs=5)

In [22]:
x_test=x_test.astype(float)/255
out = autoencoder.predict(x_test[:5])

for i in range(5):
    plt.figure()
    plt.imshow(out[i],cmap="gray")
    plt.show()